In [ ]:
"""
Created on Tue Apr 05 15:43 2022

Script to train DNN on prepared input

Author: @claraburgard

"""

In [ ]:
import numpy as np
import xarray as xr
from tqdm.notebook import trange, tqdm
import glob
import matplotlib as mpl
import seaborn as sns
import datetime
import time

from dask import delayed

import distributed

import tensorflow as tf
from tensorflow import keras
from contextlib import redirect_stdout

from basal_melt_neural_networks.constants import *
import basal_melt_neural_networks.diagnostic_functions as diag
import basal_melt_neural_networks.data_formatting as dfmt

In [ ]:
client = distributed.Client(n_workers=8, dashboard_address=':8795', local_directory='/tmp', memory_limit='4GB')

READ IN DATA

In [ ]:
run_list = ['OPM006', 'OPM016', 'OPM018', 'OPM031-2']
TS_input = 'whole_prof' #'whole_prof' #'extrapolated'
isf_list = [10, 11, 12, 13, 18, 21, 22, 23, 24, 25, 30, 31, 33, 38, 39, 40, 42, 43, 44, 45, 47, 48, 51, 52, 53, 54, 55, 58, 61, 65, 66, 69, 70, 71, 73, 75]

In [ ]:
inputpath_data = '/bettik/burgardc/DATA/NN_PARAM/interim/INPUT_DATA/'
outputpath_nn_models = '/bettik/burgardc/DATA/NN_PARAM/interim/NN_MODELS/'
outputpath_doc = '/bettik/burgardc/SCRIPTS/basal_melt_neural_networks/custom_doc/'

In [ ]:
new_timetag = False
if new_timetag:
    datetag_dt = datetime.datetime.today()
    timetag_dt = datetime.datetime.now()
    timetag = str(datetag_dt.year)+str(datetag_dt.month).zfill(2)+str(datetag_dt.day).zfill(2)+'-'+str(timetag_dt.hour).zfill(2)+str(timetag_dt.minute).zfill(2)
else:
    #timetag = '20220422-1055'
    timetag = '20220425-1050'

new_path_model = outputpath_nn_models+timetag+'/'
if not os.path.isdir(new_path_model):
    print("I did not find this folder ("+timetag+") in model folder so I created a new one, I hope that's ok!")
    os.mkdir(new_path_model)
else:
    print("This folder ("+timetag+") in model folder exists already!")

new_path_doc = outputpath_doc+timetag+'/'
if not os.path.isdir(new_path_doc):
    print("I did not find this folder ("+timetag+") in doc folder so I created a new one, I hope that's ok!")
    os.mkdir(new_path_doc)
else:
    print("This folder ("+timetag+") in doc folder exists already!")
    
new_path_input = inputpath_data+timetag+'/'
if not os.path.isdir(new_path_input):
    print("I did not find this folder ("+timetag+") in input folder so I created a new one, I hope that's ok!")
    os.mkdir(new_path_input)
else:
    print("This folder ("+timetag+") in input folder exists already!")

In [ ]:
if TS_input == 'whole_prof':
    csv_path = inputpath_data + 'WHOLE_PROF/'
elif TS_input == 'extrapolated':
    csv_path = inputpath_data + 'EXTRAPOLATED_ISFDRAFT/'

all_input_df = None        
    
for nemo_run in run_list:
    
    for kisf in tqdm(isf_list): 
    #for kisf in tqdm([23,66]): 
    #for kisf in tqdm([23]): 
        clean_df_nrun_kisf = pd.read_csv(csv_path + 'dataframe_input_isf'+str(kisf).zfill(3)+'_'+nemo_run+'.csv',index_col=[0,1,2])
        clean_df_nrun_kisf.reset_index(drop=True, inplace=True)
        clean_ds_nrun_kisf = clean_df_nrun_kisf.to_xarray()

        if all_input_df is None:
            all_input_df = clean_ds_nrun_kisf.copy()
        else:
            #print(clean_ds_nrun_kisf.index)
            #print(all_input_df.index.max())
            new_index = clean_ds_nrun_kisf.index.values + all_input_df.index.max().values+1
            #print(new_index)
            clean_ds_nrun_kisf = clean_ds_nrun_kisf.assign_coords({'index': new_index})
            all_input_df = xr.concat([all_input_df, clean_ds_nrun_kisf], dim='index') 

all_input_df.to_netcdf(new_path_input + 'dataset_input_'+timetag+'.nc','w')

DIVIDE INTO TRAIN AND TEST DATASET

In [ ]:
all_input_df = xr.open_mfdataset(new_path_input + 'dataset_input_'+timetag+'.nc')#.load()#.chunk({'index': 10000})

In [ ]:
sample_frac = 0.7

all_indexes = all_input_df.index
random_sample = np.random.choice(all_indexes, size=np.round(len(all_indexes)*0.7).astype(int), replace=False)

data_train = all_input_df.sel(index=random_sample)
data_test = all_input_df.drop_sel(index=random_sample)

In [ ]:
def print_shape_xr_ds(ds):
    return (len(ds.index),len(ds.data_vars.keys()))

In [ ]:
y_train = data_train['melt_m_ice_per_y']
x_train = data_train.drop_vars(['melt_m_ice_per_y'])

y_test = data_test['melt_m_ice_per_y']
x_test = data_test.drop_vars(['melt_m_ice_per_y'])

print('Original data shape was : ', print_shape_xr_ds(all_input_df))
print('x_train : ',print_shape_xr_ds(x_train), 'y_train : ',len(y_train))
print('x_test  : ',print_shape_xr_ds(x_test),  'y_test  : ',len(y_test))

DATA NORMALIZATION

In [ ]:
norm_method = 'interquart' #'std', 'interquart', 'minmax'

x_mean = x_train.mean()
y_mean = y_train.mean()

if norm_method == 'std':
    x_range  = x_train.std()
    y_range  = y_train.std()
elif norm_method == 'interquart':
    x_range  = x_train.quantile(0.9) - x_train.quantile(0.1)
    y_range  = y_train.quantile(0.9) - y_train.quantile(0.1)
elif norm_method == 'minmax':
    x_range  = x_train.max() - x_train.min() 
    y_range  = y_train.max() - y_train.min() 
    
x_train_norm = (x_train - x_mean)/x_range
x_test_norm = (x_test - x_mean)/x_range

y_train_norm = (y_train - y_mean)/y_range
y_test_norm = (y_test - y_mean)/y_range

In [ ]:
mean_ds = xr.merge([x_mean,y_mean])
mean_ds = mean_ds.assign_coords({'metric': 'mean_vars'})
range_ds = xr.merge([x_range,y_range])
range_ds = range_ds.assign_coords({'metric': 'std_vars'})
summary_ds = xr.concat([mean_ds, range_ds], dim='metric')

summary_ds.to_netcdf(new_path_model + 'dataset_norm_training_factors_'+timetag+'.nc')

In [ ]:
data_train_norm = xr.merge([x_train_norm,y_train_norm])
data_train_norm.to_netcdf(new_path_model + 'dataset_norm_training_data_'+timetag+'.nc', 'w')
data_test_norm = xr.merge([x_test_norm,y_test_norm])
data_test_norm.to_netcdf(new_path_model + 'dataset_norm_test_data_'+timetag+'.nc', 'w')

In [ ]:
with open(new_path_doc+'info_'+timetag+'.log','w') as file:
    file.write('Timetag: '+timetag+' \n')
    file.write('----- DATA ----- \n')
    file.write('Taraining data from: '+str(run_list)+'\n')
    file.write('Ice shelves: '+str(isf_list)+'\n')
    file.write('Input T and S format: '+TS_input+'\n')
    file.write('Norm method: '+norm_method+'\n')
    #file.write('Original data shape was : '+str(all_input_df.shape)+'\n')
    file.write('x_train : '+str(print_shape_xr_ds(x_train_norm))+', y_train : '+str(y_train_norm.values.shape)+'\n')
    file.write('x_test  : '+str(print_shape_xr_ds(x_test_norm))+', y_test  : '+str(y_test_norm.values.shape)+'\n') 

BUILD THE MODEL

In [ ]:
data_train_norm = xr.open_dataset(new_path_model + 'dataset_norm_training_data_'+timetag+'.nc')
data_test_norm = xr.open_dataset(new_path_model + 'dataset_norm_test_data_'+timetag+'.nc')

In [ ]:
y_train_norm = data_train_norm['melt_m_ice_per_y'].load()
x_train_norm = data_train_norm.drop_vars(['melt_m_ice_per_y']).to_array().load()

y_test_norm = data_test_norm['melt_m_ice_per_y'].load()
x_test_norm = data_test_norm.drop_vars(['melt_m_ice_per_y']).to_array().load()

In [ ]:
def get_model_v1(shape):
    
    model = keras.models.Sequential()
    model.add(keras.layers.Input(shape, name="InputLayer"))
    model.add(keras.layers.Dense(32, activation='relu', name='Dense_n1'))
    model.add(keras.layers.Dense(64, activation='relu', name='Dense_n2'))
    model.add(keras.layers.Dense(32, activation='relu', name='Dense_n3'))
    model.add(keras.layers.Dense(1, name='Output'))
    
    model.compile(optimizer = 'adam',
                  loss      = 'mse',
                  metrics   = ['mae', 'mse'] )
    return model

def get_model_v2(shape):
    
    model = keras.models.Sequential()
    model.add(keras.layers.Input(shape, name="InputLayer"))
    model.add(keras.layers.Dense(1, name='Output'))
    
    model.compile(optimizer = 'adam',
                  loss      = 'mse',
                  metrics   = ['mae', 'mse'] )
    return model

TRAIN THE MODEL

In [ ]:
input_size = x_train_norm.values.shape[0]

In [ ]:
model=get_model_v1( (input_size,) )

model.summary()

In [ ]:
epoch_nb = 100
batch_siz = 1024

In [ ]:
with open(new_path_doc+'info_'+timetag+'.log','a') as file:
    file.write('Input variables: '+','.join(map(str,x_train_norm['variable'].values))+'\n')
    file.write('\n')
    file.write('----- MODEL ----- \n')
    with redirect_stdout(file):
        model.summary()
    file.write('\n')
    file.write('----- TRAINING ----- \n')
    file.write('Epochs: '+str(epoch_nb)+'\n')
    file.write('Batch size: '+str(batch_siz))

In [ ]:
#reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
#                              patience=5, min_lr=0.001)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=2, min_lr=0.00000015, min_delta=0.007)

time_start = time.time()
history = model.fit(x_train_norm.T.values,
                    y_train_norm.values,
                    epochs          = epoch_nb,
                    batch_size      = batch_siz,
                    verbose         = 1,
                    validation_data = (x_test_norm.T.values, y_test_norm.values),
                   callbacks=[reduce_lr])
time_end = time.time()
timelength = time_end - time_start
with open(new_path_doc+'info_'+timetag+'.log','a') as file:
    file.write('\n Training time (in s): '+str(timelength))
model.save(new_path_model + 'model_nn_'+timetag+'.h5')

In [ ]:
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history.history) 

hist_csv_file = new_path_model+'history_'+timetag+'.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:
########### FROM THERE ON JUST TRYING AROUND

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)

In [ ]:
y_train_norm

In [ ]:
x_train_norm.to_array()

In [ ]:
y_train_norm

In [ ]:
tf.data.Dataset.

In [ ]:
x_test_norm = x_test_norm.to_array().load()

In [ ]:
y_test_norm = y_test_norm.load()

In [ ]:
x_test_norm.T.values.shape

In [ ]:
xr.apply_ufunc(model.fit,
                x_train_norm.to_array().chunk({'index': 1000,'variable': 148}), #.expand_dims('other')
                y_train_norm.chunk({'index': 1000}), #.expand_dims('other')
                kwargs={'epochs': epoch_nb, 
                        'batch_size': batch_siz,
                        'verbose': 1,
                        'validation_data': (x_test_norm.T.values, y_test_norm.values), #.expand_dims('other')
                        'callbacks': [reduce_lr]},
                
               input_core_dims=[["variable", "index"],["index", ]],
               output_core_dims=[["index" ,]],
               dask='allowed')

#

QUICK EVALUATION

In [ ]:
score = model.evaluate(x_test_norm.T.values, y_test_norm.values, verbose=1)

print('x_test / loss      : {:5.4f}'.format(score[0]))
print('x_test / mae       : {:5.4f}'.format(score[1]))
print('x_test / mse       : {:5.4f}'.format(score[2]))

In [ ]:
print("min( val_mae ) : {:.4f}".format( min(history.history["val_mae"]) ) )

In [ ]:
diag.plot_history(history, plot={'MSE' :['mse', 'val_mse'],
                                'MAE' :['mae', 'val_mae'],
                                'LOSS':['loss','val_loss']})

In [ ]:
all_input_df = None

for nemo_run in run_list:
    print(nemo_run)
    
    # read in the file
    clean_df_nrun = pd.read_csv(new_path_model + 'dataframe_input_'+nemo_run+'.csv',index_col=[0,1,2])
    clean_df_nrun.reset_index(drop=True, inplace=True)
    
    # append the different runs
    if all_input_df is None:
        all_input_df = clean_df_nrun
    else:
        all_input_df = all_input_df.append(clean_df_nrun, ignore_index = True)

In [ ]:
if TS_input == 'whole_prof':
    csv_path = inputpath_data + 'WHOLE_PROF/'
elif TS_input == 'extrapolated':
    csv_path = inputpath_data + 'EXTRAPOLATED_ISFDRAFT/'

for nemo_run in run_list:
    
    for kisf in isf_list:
        
        # copy to folder for archive
        src = csv_path + 'dataframe_input_'+str(kisf).zfill(3)+'_'+nemo_run+'.csv'
        dst = new_path_input + 'dataframe_input_'+nemo_run+'.csv'
        os.popen(f"cp {src} {dst}")
    
        print('copied '+TS_input+' input data from '+str(kisf)+' to '+nemo_run+' input folder')
